# Introduction to Bedrock - Generating images using Stable Diffusion

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a image generation task. We show how to use The Stable Diffusion foundational model to create images

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Image Creation](#2.-Generate-Images-with-Text)
3. [Image to Image](#3.-Image-to-Image) 

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

In [ ]:
%pip install -U boto3 botocore --quiet --force-reinstall

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [ ]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [2]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False  # Needed for list_foundation_models() from the control plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)


#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [3]:
bedrock_client.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'aadea722-db5d-4103-9b53-addce811b9b4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 09 Oct 2023 13:15:02 GMT',
   'content-type': 'application/json',
   'content-length': '5729',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'aadea722-db5d-4103-9b53-addce811b9b4'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium',
   'modelName': 'Titan Text Embeddings',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities'

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Image  Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId=model_id,
   contentType="application/json",
   accept="application/json",
   body=body
}

```

## 2. Generate Images with Text

#### Let's now try out the Stable Diffusion model in Amazon Bedrock to generate some images with text

In [ ]:
bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True # Default. Needed for invoke_model() from the data plane
)

In [ ]:
prompt_data ="Golden retriever playing catch at a tropical, sunny beach with palm trees in the background."

In [ ]:
negative_prompts = [
    "3d render", "smooth", "plastic", "blurry", "grainy", 
    "low-resolution", "anime", "deep-fried", "oversaturated", 
    "poorly rendered", "poor background details", "poorly drawn dog",
    "disfigured dog features"
    ]

In [ ]:
style_preset = "photographic"

In [ ]:
from botocore.config import Config
from botocore.exceptions import ClientError
import json
from PIL import Image
from io import BytesIO
import base64
from base64 import b64encode
from base64 import b64decode
import boto3


body = json.dumps(
    {"text_prompts":[{"text":prompt_data}],
     "style_preset": style_preset,
     "negative_prompts": negative_prompts
    }
) 

modelId = 'stability.stable-diffusion-xl'
accept = 'application/json'
contentType = 'application/json'

In [ ]:
response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response = json.loads(response.get('body').read())
images = response.get('artifacts')

image = Image.open(BytesIO(b64decode(images[0].get('base64'))))
image.save("generated_image.png")
display(image)

## 3. Image to Image

#### Stable Diffusion let's us do some interesting stuff with our images like adding new characters or modifying scenery let's give it a try

In [ ]:
prompt_data = "Change the dog to be a poodle"

In [ ]:
buffer = BytesIO()
img = Image.open("generated_image.png")
img.save(buffer, format="PNG")
img_bytes = buffer.getvalue()

body = json.dumps(
    {"text_prompts":[{"text": prompt_data }],
     "init_image": base64.b64encode(img_bytes).decode(),
     "style_preset": style_preset,
     "negative_prompts": negative_prompts
    }
)
modelId = 'stability.stable-diffusion-xl'

try: 
    response = bedrock_client.invoke_model(body=body, modelId=modelId, contentType="application/json", accept="image/png")
except ClientError as error:
    print(error.response)

if response['contentType'] == 'image/png':
    # Get the response body as bytes
    image_data = response['body'].read()
else:
    image_data = response['body']

image = Image.open(BytesIO(image_data))
file_name = 'converted.png'
#save file
image.save(file_name)
display(image)

## Create different image from Golden Retriever

In [ ]:
prompt_data = "Remove the ball from the image"

In [ ]:
buffer = BytesIO()
img = Image.open("generated_image.png")
img.save(buffer, format="PNG")
img_bytes = buffer.getvalue()

body = json.dumps(
    {"text_prompts":[{"text": prompt_data }],
     "init_image": base64.b64encode(img_bytes).decode(),
     "style_preset": style_preset,
     "negative_prompts": negative_prompts
    }
)
modelId = 'stability.stable-diffusion-xl'

try: 
    response = bedrock_client.invoke_model(body=body, modelId=modelId, contentType="application/json", accept="image/png")
except ClientError as error:
    print(error.response)

if response['contentType'] == 'image/png':
    # Get the response body as bytes
    image_data = response['body'].read()
else:
    image_data = response['body']

image = Image.open(BytesIO(image_data))
file_name = 'converted-to-mountains.png'
#save file
image.save(file_name)
display(image)

In [ ]:
prompt_data = "Upscale to 8K resolution"

In [ ]:
buffer = BytesIO()
img = Image.open("generated_image.png")
img.save(buffer, format="PNG")
img_bytes = buffer.getvalue()

body = json.dumps(
    {"text_prompts":[{"text": prompt_data }],
     "init_image": base64.b64encode(img_bytes).decode(),
     "style_preset": style_preset,
     "negative_prompts": negative_prompts
    }
)
modelId = 'stability.stable-diffusion-xl'

try: 
    response = bedrock_client.invoke_model(body=body, modelId=modelId, contentType="application/json", accept="image/png")
except ClientError as error:
    print(error.response)

if response['contentType'] == 'image/png':
    # Get the response body as bytes
    image_data = response['body'].read()
else:
    image_data = response['body']

image = Image.open(BytesIO(image_data))
file_name = 'upscaled-to-8k.png'
#save file
image.save(file_name)
display(image)